# Gerenciamento de tráfego - parte 2

Nesta seção iremos abordar mais formas de gerenciar o tráfego com o Istio, até o momento vimos basicamente uma formas, o roteamento por nome do host (FQDN).

Nosso novo cenário a nossa aplicação irá expor os serviços de _login_ e _order_ e algumas das suas APIs para outras aplicações.

Não precisamos expor cada um dos serviços da nossa aplicação, podemos criar um ou mais _VirtualServices_ que representem esse cenário:

| url interna               | url externa                                | service     | port |
| ---                       | ---                                        | ---         | ---  |
|                           | https://www.simul-shop.com                 | front-end   | 8000 |
|                           | https://www.simul-shop.com/login           | login       | 8000 |
|                           | https://www.simul-shop.com/catalogue       | catalogue   | 8000 |
| http://orders             | https://www.simul-shop.com/order           | orders      | 8000 |
| http://shipping           | https://www.simul-shop.com/shipping        | shipping    | 8000 |
| http://cart               | https://www.simul-shop.com/cart            | cart        | 8000 |
| http://payment            | https://www.simul-shop.com/payment         | payment     | 8000 |
| http://accounts           | https://www.simul-shop.com/accounts        | accounts    | 8000 |
| http://orders/db          |                                            | orders-db   | 8000 |
| http://queue              |                                            | queue       | 8000 |
| http://cart/db            |                                            | cart-db     | 8000 |
| http://accounts/db        |                                            | accounts-db | 8000 |

Mantenha um terminal aberto e o Kiali, iremos utilizá-los com frequência.

### Versionamento de recursos web

Embora esteja fora do escopo desse curso, essa é uma discussão que em algum momento do desenvolvimento do sistema você terá que enfrentar, como indicar para os consumidores a versão da sua aplicação.

Podemos separar o tema em duas partes:

* Interface com usuário final
* APIs


#### Interface web

Não é interessante apresentar qualquer forma de versionamento para o usuário final, ou seja, não deveríamos expor qustões internas do sistemas, como portas, protocolos e versões para os usuários finais.

Também dificultaria testes A/B, implantações canário, ou qualquer forma de manipulação das requisições que necessitasse de transparência para o consumidor.

#### APIs

É uma estória completamente diferente, os consumidores destes recursos procuram algum tipo de garantia de que as versões que utilizam são estáveis, que o contrato que estabeleceram com o produtor não seja alterado para aquele ponto no tempo.

Existem prós e contras para cada escolha no forma de versionamento, entre elas:
    
* **Controle de versão por meio de caminho na URI**: http://www.simul-shop.com/orders/api/v1
    * Prós: os clientes podem armazenar recursos em cache facilmente
    * Contras: esta solução tem ramificações na base de código
* **Controle de versão por meio de parâmetros de consulta**: http://www.simul-shop.com/orders/api?version=v1
    * Prós: é uma maneira direta de criar uma versão de API e é fácil padronizar para a versão mais recente
    * Contras: os parâmetros de consulta são mais difíceis de usar para rotear solicitações para a versão adequada da API
* **Controle de versão por meio de cabeçalhos personalizados**: `curl -H “Accept-versions: v1” http://www.simul-shop.com/orders/api`
    * Prós: não confunde o URI com informações de versão
    * Contras: requer cabeçalhos personalizados
* **Controle de versão por meio de negociação de conteúdo**: `curl -H “Accept: application/vnd.xm.device+json; version=v1” http://www.simul-shop.com/orders/api`
    * Prós: nos permite criar versões de uma representação de recurso único em vez de criar versões de toda a API, o que nos dá um controle mais granular sobre as versões. Não requer a implementação de regras de roteamento de URI.
    * Contras: Exigir cabeçalhos HTTP com tipos de mídia torna mais difícil testar e explorar a API usando um navegador.

Do ponto de vista do design da APIs, incorporar a versão no URL não é uma prática recomendada porque resulta mudanças nas URLs, mesmo que o próprio recurso não tenha mudado.

[Roy Fielding](https://en.wikipedia.org/wiki/Roy_Fielding), um dos principais contribuidores para o protocolo HTTP e criador to REST, não recomenta utilizar qualquer forma de controle de versão para APIs ([Apresentação](https://www.slideshare.net/evolve_conference/201308-fielding-evolve)).

#### Por que não versionamento semântico?

De fato, a maioria das APIs utiliza o [versionamento semântico](https://semver.org/), mas não a versão completamente, pelo menos não na URI. Nada impediria que suas URL fossem `https://www.simul-shop.com/cart/api/1.0.2', porém o número da versão principal é o que você precisa para indicar aos seus consumidores a compatibilidade entre as versões. Os números que representam as versões menores e as correções não deveriam quebrar a compatibilidade e não justificariam a mudança da URL.

Se você precisa expressar a versão completa para seus consumidores, pode utilizar os cabeçalhos personalizados, como por exemplo o `Accept-versions` ou `Content-versions`.

#### Considerações

Em resumo, quando tratamos de interface web, geralmente procuramos ser o mais transparente o possível para o usuário final, mas se a questão é APIs, nossos consumidores, geralmente outros técnicos, necessitam de alguma garantia que as mesmas versões geram os mesmos resultados. Isso pode dificultar o uso de implementação canário por uma questão de semântica.

Felizmente, o Istio fornece recursos para lidar com todas as opções, então nós optamos por indicar a versão no cabeçalho, quando necessário.

Também optamos por não adicionar a palavra `api` na URL, por uma simples questão, ela não contribui para os nossos exemplos e você pode optar em criar seus serviços com interfaces web (micro front-ends) e programáticas (APIs), sendo assim, você poderia adicionar a palavra depois do roteamento do Istio, mas isso é com você.

## Rota baseada no caminho

![rota baseada no caminha](media/path-based-routes.png)

Vamos implementar a tabela acima, já vimos como configurar _VirtualService_ para hosts, vamos configurá-lo para caminho, a estrutura para as URIs do front-end

| url interna               | url externa                                | service     | port |
| ---                       | ---                                        | ---         | ---  |
|                           | https://www.simul-shop.com                 | front-end   | 8000 |
|                           | https://www.simul-shop.com/login           | login       | 8000 |
|                           | https://www.simul-shop.com/catalogue       | catalogue   | 8000 |


```yaml
  hosts:
  - front-end # <--- FQDN completo ou abreviado
  http:
  - match:
    - uri:
        prefix: "/login"
    rewrite:
      uri: "/"
    route:
    - destination:
        host: login # FQDN completo ou abreviado
        subset: v1
  - match:
        prefix: "/catalogue"
    rewrite:
      uri: "/"
    route:
    - destination:
        host: catalogue # FQDN completo ou abreviado
        subset: v1
  - route: # <-- rota padrão
    - destination:
        host: front-end 
        subset: v1
```

## Rota baseada no cabeçalho

TODO

## Multiplas regras de tráfego

TODO

## Modificando os cabeçalhos de resposta

TODO

## Retentativas e timeouts

TODO

## Interrupção de circuíto

TODO

## Espelhando o tráfego

TODO